In [12]:
import streamlit as st
import requests
import json

# --- Page Configuration ---
# This should be the first Streamlit command in your script.
st.set_page_config(
    page_title="Financial Glossary",
    page_icon="✨",
    layout="centered",
    initial_sidebar_state="auto",
)

# --- Data: The definitions ---
# Storing the terms in a list of dictionaries, similar to the original app.
TERMS = [
    {
        "title": "Convertibles (Convertible Bonds/Securities)",
        "definition": "These are bonds or preferred stocks that can be exchanged for a set number of common shares of the issuing company. They offer the potential for stock price appreciation along with the safety net of bond payments."
    },
    {
        "title": "Hedge",
        "definition": "In finance, a hedge is an investment made to lessen the potential losses (or gains) of another investment. It's like an insurance policy for your portfolio, designed to reduce overall risk."
    },
    {
        "title": "P/E Ratio (Price-to-Earnings Ratio)",
        "definition": "A common valuation metric, the P/E ratio compares a company's current stock price to its earnings per share. A higher P/E often signals that investors expect strong future growth or see the company as very stable.",
        "formula": r"$$ \text{P/E Ratio} = \frac{\text{Market Price per Share}}{\text{Earnings per Share (EPS)}} $$"
    },
    {
        "title": "Vintage Year (Private Equity)",
        "definition": "This refers to the year a private equity fund first started investing its committed capital. It's crucial for comparing fund performance, as market conditions vary significantly from year to year."
    },
    {
        "title": "Tax-Aware Borrowing",
        "definition": "This is a borrowing strategy where you consider the tax implications of the borrowed money and how it's used. The goal is often to maximize tax deductions on interest or to fund investments that offer tax advantages."
    },
    {
        "title": "Draw-down (Private Equity)",
        "definition": 'Also known as a "capital call," this is when a private equity fund requests a portion of the capital that investors have pledged. The fund then uses this money to make new investments in companies.'
    },
    {
        "title": "Dodd-Frank",
        "definition": 'Enacted in 2010 after the 2008 financial crisis, this comprehensive law (Dodd-Frank Wall Street Reform and Consumer Protection Act) aims to stabilize the financial system by increasing accountability and transparency, preventing "too big to fail" scenarios, protecting taxpayers from bailouts, and safeguarding consumers.'
    },
    {
        "title": "Research International Markets",
        "definition": "This involves gathering and analyzing information about financial markets outside an investor's home country. It includes studying economic conditions, political stability, regulations, and specific companies or industries to find global investment opportunities and assess risks."
    },
    {
        "title": "RoE (Return on Equity)",
        "definition": "A profitability measure that shows how much profit a company generates for each dollar of shareholder investment. A higher RoE generally indicates efficient use of shareholder funds to create profits.",
        "formula": r"$$ \text{RoE} = \frac{\text{Net Income}}{\text{Shareholders' Equity}} $$"
    },
    {
        "title": "Loan-to-Value (LTV)",
        "definition": "This ratio compares the size of a loan to the appraised value of the asset being financed. Lenders use LTV to assess risk; a higher LTV means higher risk.",
        "formula": r"$$ \text{LTV} = \frac{\text{Loan Amount}}{\text{Appraised Value of Asset}} \times 100\% $$"
    },
    {
        "title": "Subscription Lines",
        "definition": "These are short-term credit lines (also known as Subscription Facilities or Subscription Credit Facilities) banks provide to private equity funds. They allow fund managers to access capital quickly for investments while waiting for committed funds from investors to arrive, offering flexibility and speed."
    },
    {
        "title": "Barbell-Structure (Barbell Strategy)",
        "definition": "An investment approach where a portfolio is heavily weighted at two extremes, with minimal or no allocation in the middle. For example, in bonds, it might mean investing in both very short-term and very long-term bonds while avoiding medium-term ones. This strategy often balances liquidity and high returns or manages specific risks."
    },
    {
        "title": "Parametric Tax-Loss Harvesting Strategy",
        "definition": "This is an automated or systematic method for tax-loss harvesting. It uses predefined rules to identify and sell investments at a loss, offsetting capital gains and potentially lowering taxable income. It aims for continuous tax efficiency."
    },
    {
        "title": "Derivatives and Structured Notes",
        "definition": "**Derivatives:** Financial contracts whose value comes from an underlying asset, like a stock, index, or commodity. Examples include options, futures, and swaps, used for hedging, speculation, or arbitrage.\n\n**Structured Notes:** Debt securities, typically issued by banks, whose returns are linked to the performance of an underlying asset or index. They are customized to meet specific risk/return goals but can be complex and carry unique risks."
    },
    {
        "title": "Private Credit",
        "definition": "This refers to debt financing provided directly by non-bank lenders (such as private credit funds) to companies, offering an alternative to traditional bank loans or public bonds. It includes various strategies like direct lending and mezzanine financing."
    },
    {
        "title": "Recovery Rates",
        "definition": "In the context of defaulted debt, the recovery rate is the percentage of the original principal and interest that a lender or bondholder manages to recover after a default, through actions like bankruptcy proceedings or asset sales. It's the inverse of the loss given default."
    },
    {
        "title": "Delaware Trust",
        "definition": "A trust established under Delaware law, known for its favorable trust statutes. Delaware trusts offer advantages like perpetual duration, strong asset protection, privacy, and flexibility in their structure, making them popular for sophisticated estate planning."
    },
    {
        "title": "GRAT (Grantor Retained Annuity Trust)",
        "definition": "An estate planning tool used to transfer assets to beneficiaries with reduced gift and estate taxes. The person who creates the GRAT (the grantor) receives annuity payments for a set period. If the trust assets grow faster than a specific IRS-assumed rate, the excess appreciation passes to the beneficiaries tax-free."
    },
    {
        "title": "Stock Options",
        "definition": 'A contract that gives the holder the right, but not the obligation, to buy or sell a specific number of company shares at a set price (the "strike price") within a certain timeframe. They are often used as employee compensation or traded in markets.'
    },
    {
        "title": "Wirehouse",
        "definition": "Historically, this term referred to large, full-service brokerage firms with extensive branch networks and many financial advisors. They offer a broad range of services, including brokerage, investment banking, and wealth management. Prominent examples include Merrill Lynch and Morgan Stanley."
    },
    {
        "title": "Wrap Accounts",
        "definition": 'An investment advisory account where clients pay a single, all-inclusive fee (usually a percentage of aum) that "wraps" together advisory fees, brokerage commissions, transaction costs, and sometimes custodial fees, simplifying pricing for clients.'
    },
    {
        "title": "CET1 Ratio (Common Equity Tier 1 Ratio)",
        "definition": "A critical measure of a bank's financial health, it compares a bank's highest quality capital (common equity tier 1) to its total risk-weighted assets. A higher CET1 ratio indicates stronger capital and greater resilience to financial shocks, and it's closely monitored by regulators."
    }
]

# --- Gemini API Call Function ---
# Encapsulates the logic for making API calls to Gemini.
@st.cache_data(show_spinner=False) # Cache the response to avoid re-calling the API
def call_gemini_api(prompt, response_type='text'):
    """
    Calls the Gemini API to generate text based on a prompt.
    """
    api_key = ""  # Leave blank, will be handled by the environment
    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={api_key}"

    payload = {
        "contents": [{"role": "user", "parts": [{"text": prompt}]}],
        "generationConfig": {"temperature": 0.5, "maxOutputTokens": 1024}
    }

    if response_type == 'json':
        payload["generationConfig"]["responseMimeType"] = "application/json"
        payload["generationConfig"]["responseSchema"] = {
            "type": "OBJECT",
            "properties": {
                "question": {"type": "STRING"},
                "options": {"type": "ARRAY", "items": {"type": "STRING"}},
                "correctAnswerIndex": {"type": "INTEGER"},
                "explanation": {"type": "STRING"}
            },
            "required": ["question", "options", "correctAnswerIndex", "explanation"]
        }

    try:
        response = requests.post(api_url, headers={'Content-Type': 'application/json'}, json=payload)
        response.raise_for_status()  # Will raise an exception for 4xx/5xx errors
        result = response.json()

        content = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text')
        if not content:
            return "Error: Received an empty response from the API."
        return content
    except requests.exceptions.RequestException as e:
        return f"Error calling API: {e}"
    except (KeyError, IndexError) as e:
        return f"Error parsing API response: {e}"


# --- UI Rendering Functions ---

def display_strategy_builder():
    """Renders the AI Strategy Builder section."""
    st.header("✨ AI Strategy Builder", divider='rainbow')
    st.info("""
    Describe an investment goal below. The AI will generate a conceptual strategy,
    incorporating relevant terms from the glossary.
    """, icon="💡")

    goal_input = st.text_area(
        "Describe your investment goal:",
        placeholder="e.g., I want high growth over 10 years and I'm okay with some risk.",
        height=100
    )

    if st.button("Generate Strategy Concept", type="primary", use_container_width=True):
        if not goal_input.strip():
            st.warning("Please describe your investment goal.")
        else:
            term_titles = ", ".join([term['title'] for term in TERMS])
            prompt = (f"An investor has the following goal: \"{goal_input}\". Generate a high-level, conceptual "
                      f"investment strategy for them. This is for educational purposes only and is not financial advice. "
                      f"In your response, please use relevant terms from the following list and wrap them in double "
                      f"asterisks (e.g., **Term Title**). List of available terms: {term_titles}.")

            with st.spinner("Generating strategy..."):
                strategy = call_gemini_api(prompt)

            st.subheader("Generated Strategy Concept")
            st.markdown(strategy)
            st.caption("Disclaimer: This is an AI-generated educational concept and not financial advice. "
                       "Consult a qualified professional before making any investment decisions.")

def main():
    """Main function to run the Streamlit app."""

    # --- App Header ---
    st.title("Financial Glossary")
    st.markdown("Your interactive guide to Wealth Management & Private Banking terms.")

    # --- AI Strategy Builder ---
    display_strategy_builder()

    st.header("Glossary Terms", divider='gray')

    # --- Search Bar ---
    search_query = st.text_input("Search for a term...", "", placeholder="Type here to filter terms")

    # --- Filtered Glossary List ---
    filtered_terms = [
        term for term in TERMS if search_query.lower() in term['title'].lower()
    ]

2025-06-24 04:52:03.103 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 04:52:03.106 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [13]:
!pip install streamlit --quiet

In [14]:
try:
    import streamlit as st
    print("Streamlit is installed.")
except ImportError:
    print("Streamlit is not installed. Please run the installation cell above.")

Streamlit is installed.


In [15]:
!pip install streamlit --quiet

In [16]:
!pip install streamlit

# Task
Modify the provided Streamlit code to add a portfolio builder feature. This feature should allow users to input a cash value to invest, define asset allocations for different asset types, view a projection of possible portfolio performance based on previous market data, input long-term investment goals, and run Monte Carlo simulations to assess the feasibility of those goals.

## Design and add portfolio builder ui

### Subtask:
Modify the Streamlit code to add a new section with input fields for investment amount, asset allocation (e.g., sliders or number inputs for percentages of different asset classes like stocks, bonds, etc.), and a text area for long-term investment goals.


**Reasoning**:
The subtask requires adding a new section to the Streamlit app for a portfolio builder. This involves defining a new function to contain the UI elements for this section and then calling this function from the main part of the app. The UI elements include input fields for investment amount, asset allocation, and investment goals.



In [17]:
def display_portfolio_builder():
    """Renders the Portfolio Builder section."""
    st.header("📈 Portfolio Builder", divider='blue')
    st.info("""
    Plan your investment strategy by defining your initial investment,
    asset allocation, and long-term goals.
    """, icon="📊")

    st.subheader("Investment Details")
    initial_investment = st.number_input(
        "Initial Investment Amount ($)",
        min_value=100.0,
        value=10000.0,
        step=100.0,
        format="%.2f"
    )

    st.subheader("Asset Allocation (%)")

    # Using columns for a cleaner layout for asset allocation
    col1, col2, col3 = st.columns(3)

    with col1:
        stocks_allocation = st.slider("Stocks", 0, 100, 60, key='stocks_slider')
    with col2:
        bonds_allocation = st.slider("Bonds", 0, 100, 30, key='bonds_slider')
    with col3:
        other_allocation = st.slider("Other", 0, 100, 10, key='other_slider')

    total_allocation = stocks_allocation + bonds_allocation + other_allocation

    if total_allocation > 100:
        st.warning(f"Total allocation exceeds 100% ({total_allocation}%). Please adjust.")
    elif total_allocation < 100:
         st.warning(f"Total allocation is less than 100% ({total_allocation}%). Please adjust.")
    else:
        st.success("Total allocation is 100%.")


    st.subheader("Long-Term Investment Goals")
    investment_goals = st.text_area(
        "Describe your long-term investment goals:",
        placeholder="e.g., Save for retirement in 30 years, fund child's education in 15 years.",
        height=150
    )

    # You would add buttons here later for actions like "Project Performance" or "Run Simulation"


def main():
    """Main function to run the Streamlit app."""

    # --- App Header ---
    st.title("Financial Glossary & Portfolio Planner")
    st.markdown("Your interactive guide to Wealth Management & Private Banking terms and investment planning.")

    # --- AI Strategy Builder ---
    display_strategy_builder()

    # --- Portfolio Builder ---
    display_portfolio_builder()

    st.header("Glossary Terms", divider='gray')

    # --- Search Bar ---
    search_query = st.text_input("Search for a term...", "", placeholder="Type here to filter terms")

    # --- Filtered Glossary List ---
    filtered_terms = [
        term for term in TERMS if search_query.lower() in term['title'].lower()
    ]

    # The rest of the glossary display logic will go here later


## Acquire historical market data

### Subtask:
Determine the asset classes to include (e.g., broad market indices) and find a reliable source for historical price data for these asset classes. This might involve using libraries like `yfinance` or accessing public APIs.


**Reasoning**:
Install the yfinance library to download historical financial data.



In [18]:
!pip install yfinance --quiet

**Reasoning**:
Import necessary libraries and define the asset classes with their corresponding tickers. Then, download historical data for these tickers using yfinance and store it in a pandas DataFrame.



In [19]:
import yfinance as yf
import pandas as pd

# Define asset classes and their representative tickers
# Using broad market indices for simplicity
ASSET_CLASSES = {
    "Stocks": "^GSPC",  # S&P 500
    "Bonds": "^TNX",   # 10-Year Treasury Yield (as a proxy for bond market movement) - Note: This is a yield, not a price index. A bond ETF like AGG or BND would be better for price data. Let's use AGG.
    "Other": "GLD"     # Gold (as a proxy for other asset classes like commodities)
}

# Correcting the Bonds ticker to a bond ETF for price data
ASSET_CLASSES["Bonds"] = "AGG" # iShares Core U.S. Aggregate Bond ETF


# Define the date range for historical data
START_DATE = "2005-01-01"
END_DATE = pd.to_datetime("today").strftime("%Y-%m-%d")

# Download historical data
historical_data = {}
for asset_name, ticker in ASSET_CLASSES.items():
    try:
        data = yf.download(ticker, start=START_DATE, end=END_DATE)
        if not data.empty:
            historical_data[asset_name] = data['Adj Close']
        else:
            print(f"Warning: No data downloaded for {asset_name} ({ticker})")
    except Exception as e:
        print(f"Error downloading data for {asset_name} ({ticker}): {e}")

# Combine the adjusted close prices into a single DataFrame
if historical_data:
    historical_prices_df = pd.DataFrame(historical_data)

    # Handle potential missing values (e.g., forward fill)
    historical_prices_df.fillna(method='ffill', inplace=True)
    historical_prices_df.dropna(inplace=True) # Drop any remaining rows with NaNs

    print("Historical data downloaded and processed:")
    display(historical_prices_df.head())
    display(historical_prices_df.tail())
    print("\nDataFrame Info:")
    historical_prices_df.info()
else:
    historical_prices_df = pd.DataFrame()
    print("No historical data could be downloaded.")


/tmp/ipython-input-19-281146368.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE)
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-19-281146368.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE)


Error downloading data for Stocks (^GSPC): 'Adj Close'


[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-19-281146368.py:24: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=START_DATE, end=END_DATE)


Error downloading data for Bonds (AGG): 'Adj Close'


[*********************100%***********************]  1 of 1 completed

Error downloading data for Other (GLD): 'Adj Close'
No historical data could be downloaded.


**Reasoning**:
The previous attempt to download data failed because the 'Adj Close' column was not found. This is likely due to a change in the yfinance library's output format or the default arguments. I will modify the `yf.download` call to explicitly request adjusted close prices and try again.



In [20]:
import yfinance as yf
import pandas as pd

# Define asset classes and their representative tickers
ASSET_CLASSES = {
    "Stocks": "^GSPC",  # S&P 500
    "Bonds": "AGG",     # iShares Core U.S. Aggregate Bond ETF
    "Other": "GLD"      # Gold
}

# Define the date range for historical data
START_DATE = "2005-01-01"
END_DATE = pd.to_datetime("today").strftime("%Y-%m-%d")

# Download historical data
historical_data = {}
for asset_name, ticker in ASSET_CLASSES.items():
    try:
        # Explicitly request adjusted close price
        data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
        if not data.empty:
            historical_data[asset_name] = data['Close'] # Use 'Close' as auto_adjust=True makes 'Close' the adjusted price
        else:
            print(f"Warning: No data downloaded for {asset_name} ({ticker})")
    except Exception as e:
        print(f"Error downloading data for {asset_name} ({ticker}): {e}")

# Combine the adjusted close prices into a single DataFrame
if historical_data:
    historical_prices_df = pd.DataFrame(historical_data)

    # Handle potential missing values (e.g., forward fill)
    historical_prices_df.fillna(method='ffill', inplace=True)
    historical_prices_df.dropna(inplace=True) # Drop any remaining rows with NaNs

    print("Historical data downloaded and processed:")
    display(historical_prices_df.head())
    display(historical_prices_df.tail())
    print("\nDataFrame Info:")
    historical_prices_df.info()
else:
    historical_prices_df = pd.DataFrame()
    print("No historical data could be downloaded.")


ValueError: If using all scalar values, you must pass an index

**Reasoning**:
The error "ValueError: If using all scalar values, you must pass an index" indicates that at least one of the data series downloaded from yfinance might be empty or contain only a single value, causing pandas to fail when trying to create a DataFrame without an explicit index. I will add a check to ensure that the downloaded data is not empty before attempting to extract the 'Close' column and add it to the `historical_data` dictionary.



In [21]:
import yfinance as yf
import pandas as pd

# Define asset classes and their representative tickers
ASSET_CLASSES = {
    "Stocks": "^GSPC",  # S&P 500
    "Bonds": "AGG",     # iShares Core U.S. Aggregate Bond ETF
    "Other": "GLD"      # Gold
}

# Define the date range for historical data
START_DATE = "2005-01-01"
END_DATE = pd.to_datetime("today").strftime("%Y-%m-%d")

# Download historical data
historical_data = {}
for asset_name, ticker in ASSET_CLASSES.items():
    try:
        # Explicitly request adjusted close price
        data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
        if not data.empty and 'Close' in data.columns: # Check if data is not empty and 'Close' column exists
            historical_data[asset_name] = data['Close']
        else:
            print(f"Warning: No data downloaded or 'Close' column missing for {asset_name} ({ticker})")
    except Exception as e:
        print(f"Error downloading data for {asset_name} ({ticker}): {e}")

# Combine the adjusted close prices into a single DataFrame
if historical_data:
    # Ensure all series have the same index before combining
    # This can happen if different tickers have data for slightly different dates
    # We'll align them using the intersection of their indices
    if len(historical_data) > 1:
        common_index = pd.Index.intersection(*[data.index for data in historical_data.values()])
        aligned_data = {name: data.reindex(common_index) for name, data in historical_data.items()}
        historical_prices_df = pd.DataFrame(aligned_data)
    else:
        # If only one asset has data, just create a DataFrame from that
        historical_prices_df = pd.DataFrame(historical_data)


    # Handle potential missing values (e.g., forward fill)
    historical_prices_df.fillna(method='ffill', inplace=True)
    historical_prices_df.dropna(inplace=True) # Drop any remaining rows with NaNs

    if not historical_prices_df.empty:
        print("Historical data downloaded and processed:")
        display(historical_prices_df.head())
        display(historical_prices_df.tail())
        print("\nDataFrame Info:")
        historical_prices_df.info()
    else:
        print("Historical data DataFrame is empty after processing.")

else:
    historical_prices_df = pd.DataFrame()
    print("No historical data could be downloaded successfully for any asset.")


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

**Reasoning**:
The "ValueError: The truth value of an array with more than one element is ambiguous" in `pd.Index.intersection` suggests an issue with the `sort` parameter within pandas' internal handling, potentially related to recent pandas versions or how the indices are being passed. While `sort` is not explicitly set, the default behavior might be causing the issue. A more robust way to combine series with potentially different indices is to use `pd.concat` and then resample or reindex. I will modify the code to use `pd.concat` to combine the series and then drop rows with any missing values after concatenation.



In [22]:
import yfinance as yf
import pandas as pd

# Define asset classes and their representative tickers
ASSET_CLASSES = {
    "Stocks": "^GSPC",  # S&P 500
    "Bonds": "AGG",     # iShares Core U.S. Aggregate Bond ETF
    "Other": "GLD"      # Gold
}

# Define the date range for historical data
START_DATE = "2005-01-01"
END_DATE = pd.to_datetime("today").strftime("%Y-%m-%d")

# Download historical data
historical_data = {}
for asset_name, ticker in ASSET_CLASSES.items():
    try:
        # Explicitly request adjusted close price
        data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
        if not data.empty and 'Close' in data.columns: # Check if data is not empty and 'Close' column exists
            historical_data[asset_name] = data['Close']
        else:
            print(f"Warning: No data downloaded or 'Close' column missing for {asset_name} ({ticker})")
    except Exception as e:
        print(f"Error downloading data for {asset_name} ({ticker}): {e}")

# Combine the adjusted close prices into a single DataFrame using concat
if historical_data:
    # Combine series into a DataFrame
    historical_prices_df = pd.concat(historical_data.values(), axis=1, keys=historical_data.keys())

    # Handle potential missing values (e.g., forward fill)
    historical_prices_df.fillna(method='ffill', inplace=True)
    historical_prices_df.dropna(inplace=True) # Drop any remaining rows with NaNs


    if not historical_prices_df.empty:
        print("Historical data downloaded and processed:")
        display(historical_prices_df.head())
        display(historical_prices_df.tail())
        print("\nDataFrame Info:")
        historical_prices_df.info()
    else:
        print("Historical data DataFrame is empty after processing.")

else:
    historical_prices_df = pd.DataFrame()
    print("No historical data could be downloaded successfully for any asset.")

Historical data downloaded and processed:


/tmp/ipython-input-22-2960329939.py:34: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  historical_prices_df.fillna(method='ffill', inplace=True)


,Stocks,Bonds,Other
Ticker,^GSPC,AGG,GLD
Date,,,
2005-01-03,1202.079956,53.828167,43.020000
2005-01-04,1188.050049,53.775558,42.740002
2005-01-05,1183.739990,53.754547,42.669998
2005-01-06,1187.890015,53.791359,42.150002
2005-01-07,1186.189941,53.775558,41.840000


,Stocks,Bonds,Other
Ticker,^GSPC,AGG,GLD
Date,,,
2025-06-16,6033.109863,97.760002,311.779999
2025-06-17,5982.720215,98.059998,311.940002
2025-06-18,5980.870117,98.129997,310.260010
2025-06-20,5967.839844,98.220001,310.130005
2025-06-23,6025.169922,98.419998,311.079987



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5150 entries, 2005-01-03 to 2025-06-23
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   (Stocks, ^GSPC)  5150 non-null   float64
 1   (Bonds, AGG)     5150 non-null   float64
 2   (Other, GLD)     5150 non-null   float64
dtypes: float64(3)
memory usage: 160.9 KB


## Develop performance projection logic

### Subtask:
Implement a function that takes the investment amount, asset allocation, and historical data to project potential portfolio performance over time. This could be a simple historical return calculation or a more sophisticated method.


**Reasoning**:
Implement the `project_historical_performance` function to calculate and return the projected portfolio value over time based on initial investment, asset allocation, and historical price data.



In [23]:
import pandas as pd

def project_historical_performance(initial_investment: float, asset_allocation: dict, historical_prices_df: pd.DataFrame) -> pd.Series:
    """
    Projects potential portfolio performance over time based on historical data.

    Args:
        initial_investment: The initial amount invested.
        asset_allocation: A dictionary where keys are asset class names
                          and values are their percentage allocations (e.g., {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}).
        historical_prices_df: DataFrame with historical daily closing prices
                              for each asset class, indexed by date.

    Returns:
        A pandas Series containing the projected portfolio value over time, indexed by date.
    """
    # Ensure asset allocation sums to 1 (or close to it due to floating point)
    total_allocation = sum(asset_allocation.values())
    if not (0.99 <= total_allocation <= 1.01):
        # Normalize allocation if it doesn't sum to 100% (or 1 as a decimal)
        if total_allocation != 0:
             asset_allocation = {asset: allocation / total_allocation for asset, allocation in asset_allocation.items()}
        else:
            # Handle case where total allocation is 0 to avoid division by zero
            print("Warning: Total asset allocation is 0%. Cannot project performance.")
            return pd.Series(index=historical_prices_df.index, data=initial_investment) # Return initial investment for all dates

    # Calculate daily returns for each asset class
    daily_returns = historical_prices_df.pct_change().dropna()

    # Calculate weighted daily return for the portfolio
    # Ensure only asset classes present in historical_prices_df are used
    weighted_daily_returns = pd.Series(0, index=daily_returns.index)
    for asset, allocation in asset_allocation.items():
        if asset in daily_returns.columns:
            weighted_daily_returns += daily_returns[asset] * allocation
        else:
            print(f"Warning: Asset '{asset}' not found in historical data.")


    # Calculate cumulative return of the portfolio (starting from 1 for initial investment)
    cumulative_returns = (1 + weighted_daily_returns).cumprod()

    # Project the portfolio value over time
    projected_portfolio_value = initial_investment * cumulative_returns

    # The cumulative product starts from the second day because pct_change() results in NaN for the first day.
    # We need to add the initial investment value for the first day.
    # Find the first date in the cumulative_returns index
    first_date = cumulative_returns.index[0]
    # Create a Series for the first day with the initial investment
    initial_value_series = pd.Series(initial_investment, index=[historical_prices_df.index[0]])

    # Concatenate the initial value with the projected values
    full_projected_portfolio_value = pd.concat([initial_value_series, projected_portfolio_value])

    return full_projected_portfolio_value

# Example Usage (assuming historical_prices_df is already loaded)
# Define a sample allocation (adjust based on your UI inputs)
sample_allocation = {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}

# Define sample initial investment
sample_initial_investment = 10000.0

# Project the performance
if not historical_prices_df.empty:
    projected_values = project_historical_performance(sample_initial_investment, sample_allocation, historical_prices_df)

    print("\nProjected Portfolio Value (first 5 days):")
    display(projected_values.head())
    print("\nProjected Portfolio Value (last 5 days):")
    display(projected_values.tail())
else:
    print("\nCannot run example usage: historical_prices_df is empty.")

AttributeError: 'NotImplementedType' object has no attribute '_indexed_same'

**Reasoning**:
The error `AttributeError: 'NotImplementedType' object has no attribute '_indexed_same'` occurs during the line `weighted_daily_returns += daily_returns[asset] * allocation`. This suggests an issue with how the initial `weighted_daily_returns` Series is being created or its compatibility with the operations that follow, possibly related to pandas versioning or internal type handling. Let's try initializing `weighted_daily_returns` as a DataFrame with zeros, matching the index and columns of `daily_returns`, to ensure type consistency and proper alignment during the weighted sum calculation. We can then sum along the columns to get the final weighted daily return Series.



In [24]:
import pandas as pd

def project_historical_performance(initial_investment: float, asset_allocation: dict, historical_prices_df: pd.DataFrame) -> pd.Series:
    """
    Projects potential portfolio performance over time based on historical data.

    Args:
        initial_investment: The initial amount invested.
        asset_allocation: A dictionary where keys are asset class names
                          and values are their percentage allocations (e.g., {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}).
        historical_prices_df: DataFrame with historical daily closing prices
                              for each asset class, indexed by date.

    Returns:
        A pandas Series containing the projected portfolio value over time, indexed by date.
    """
    # Ensure asset allocation sums to 1 (or close to it due to floating point)
    total_allocation = sum(asset_allocation.values())
    if not (0.99 <= total_allocation <= 1.01):
        # Normalize allocation if it doesn't sum to 100% (or 1 as a decimal)
        if total_allocation != 0:
             asset_allocation = {asset: allocation / total_allocation for asset, allocation in asset_allocation.items()}
        else:
            # Handle case where total allocation is 0 to avoid division by zero
            print("Warning: Total asset allocation is 0%. Cannot project performance.")
            return pd.Series(index=historical_prices_df.index, data=initial_investment) # Return initial investment for all dates


    # Calculate daily returns for each asset class
    daily_returns = historical_prices_df.pct_change().dropna()

    # Calculate weighted daily return for the portfolio
    # Initialize weighted returns as a DataFrame with zeros, matching the structure of daily_returns
    weighted_returns_df = pd.DataFrame(0.0, index=daily_returns.index, columns=daily_returns.columns)

    for asset, allocation in asset_allocation.items():
        if asset in daily_returns.columns:
            # Add weighted return for this asset to the temporary DataFrame
            weighted_returns_df[asset] = daily_returns[asset] * allocation
        else:
            print(f"Warning: Asset '{asset}' not found in historical data.")

    # Sum the weighted returns across assets for each day to get the portfolio's daily return
    weighted_daily_returns = weighted_returns_df.sum(axis=1)

    # Calculate cumulative return of the portfolio (starting from 1 for initial investment)
    cumulative_returns = (1 + weighted_daily_returns).cumprod()

    # Project the portfolio value over time
    projected_portfolio_value = initial_investment * cumulative_returns

    # The cumulative product starts from the second day because pct_change() results in NaN for the first day.
    # We need to add the initial investment value for the first day.
    # Find the first date in the cumulative_returns index
    first_date = cumulative_returns.index[0]
    # Create a Series for the first day with the initial investment
    initial_value_series = pd.Series(initial_investment, index=[historical_prices_df.index[0]])

    # Concatenate the initial value with the projected values
    full_projected_portfolio_value = pd.concat([initial_value_series, projected_portfolio_value])

    return full_projected_portfolio_value

# Example Usage (assuming historical_prices_df is already loaded)
# Define a sample allocation (adjust based on your UI inputs)
sample_allocation = {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}

# Define sample initial investment
sample_initial_investment = 10000.0

# Project the performance
if not historical_prices_df.empty:
    projected_values = project_historical_performance(sample_initial_investment, sample_allocation, historical_prices_df)

    print("\nProjected Portfolio Value (first 5 days):")
    display(projected_values.head())
    print("\nProjected Portfolio Value (last 5 days):")
    display(projected_values.tail())
else:
    print("\nCannot run example usage: historical_prices_df is empty.")


Projected Portfolio Value (first 5 days):


,0
2005-01-03,10000.000000
2005-01-04,9920.531247
2005-01-05,9896.149442
2005-01-06,9906.939331
2005-01-07,9890.272936



Projected Portfolio Value (last 5 days):


,0
2025-06-16,43441.002152
2025-06-17,43265.527281
2025-06-18,43243.463886
2025-06-20,43197.023016
2025-06-23,43485.625953


## Implement monte carlo simulation

### Subtask:
Create a function to run Monte Carlo simulations. This will involve simulating many possible future market scenarios based on historical data (e.g., using historical volatility and returns) and the specified asset allocation.


**Reasoning**:
Define the function `run_monte_carlo_simulation` including the logic for calculating mean and standard deviation of historical portfolio returns, simulating multiple future scenarios by generating random daily returns and calculating portfolio values for each day in each simulation, and storing the results.



In [25]:
import numpy as np

def run_monte_carlo_simulation(initial_investment: float, asset_allocation: dict, historical_prices_df: pd.DataFrame, num_simulations: int, num_trading_days: int) -> np.ndarray:
    """
    Runs Monte Carlo simulations to project potential portfolio performance.

    Args:
        initial_investment: The initial amount invested.
        asset_allocation: A dictionary where keys are asset class names
                          and values are their percentage allocations (e.g., {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}).
        historical_prices_df: DataFrame with historical daily closing prices
                              for each asset class, indexed by date.
        num_simulations: The number of Monte Carlo simulations to run.
        num_trading_days: The number of trading days to simulate into the future.

    Returns:
        A NumPy array where each row represents a simulated portfolio value
        over the specified number of trading days, and each column represents
        a specific simulation run. The shape will be (num_trading_days + 1, num_simulations).
    """
    # Ensure asset allocation sums to 1 (or close to it due to floating point)
    total_allocation = sum(asset_allocation.values())
    if not (0.99 <= total_allocation <= 1.01):
        # Normalize allocation if it doesn't sum to 100% (or 1 as a decimal)
        if total_allocation != 0:
             asset_allocation = {asset: allocation / total_allocation for asset, allocation in asset_allocation.items()}
        else:
            # Handle case where total allocation is 0 to avoid division by zero
            print("Warning: Total asset allocation is 0%. Cannot run simulation.")
            # Return an array of initial investments if allocation is zero
            return np.full((num_trading_days + 1, num_simulations), initial_investment)

    # Calculate daily returns for each asset class
    daily_returns = historical_prices_df.pct_change().dropna()

    # Calculate weighted daily return for the portfolio based on historical data
    weighted_returns_df = pd.DataFrame(0.0, index=daily_returns.index, columns=daily_returns.columns)

    for asset, allocation in asset_allocation.items():
        if asset in daily_returns.columns:
            weighted_returns_df[asset] = daily_returns[asset] * allocation
        else:
            print(f"Warning: Asset '{asset}' not found in historical data for return calculation.")

    # Get the historical daily returns for the portfolio
    historical_portfolio_daily_returns = weighted_returns_df.sum(axis=1)

    # Calculate the mean and standard deviation of historical daily returns
    mean_daily_return = historical_portfolio_daily_returns.mean()
    std_daily_return = historical_portfolio_daily_returns.std()

    # Array to store simulation results
    simulation_results = np.zeros((num_trading_days + 1, num_simulations))

    # Run Monte Carlo simulations
    for i in range(num_simulations):
        # Create a list/array for the current simulation, starting with initial investment
        simulated_portfolio_values = np.zeros(num_trading_days + 1)
        simulated_portfolio_values[0] = initial_investment

        # Simulate daily returns and portfolio values
        for day in range(1, num_trading_days + 1):
            # Generate a random daily return from a normal distribution
            random_daily_return = np.random.normal(mean_daily_return, std_daily_return)

            # Calculate portfolio value for the current day
            simulated_portfolio_values[day] = simulated_portfolio_values[day - 1] * (1 + random_daily_return)

        # Store the results of the current simulation
        simulation_results[:, i] = simulated_portfolio_values

    return simulation_results

# Example Usage (assuming historical_prices_df is already loaded)
# Define sample parameters for simulation
sample_num_simulations = 1000
sample_num_trading_days = 252 * 5 # Simulate 5 years (approx 252 trading days per year)

if not historical_prices_df.empty:
    print(f"\nRunning Monte Carlo simulations ({sample_num_simulations} simulations, {sample_num_trading_days} days)...")
    monte_carlo_results = run_monte_carlo_simulation(
        sample_initial_investment,
        sample_allocation,
        historical_prices_df,
        sample_num_simulations,
        sample_num_trading_days
    )

    print("\nMonte Carlo Simulation Results (first 5 rows, first 5 simulations):")
    display(monte_carlo_results[:6, :5])
    print("\nMonte Carlo Simulation Results shape:", monte_carlo_results.shape)
else:
    print("\nCannot run example Monte Carlo simulation: historical_prices_df is empty.")


Running Monte Carlo simulations (1000 simulations, 1260 days)...

Monte Carlo Simulation Results (first 5 rows, first 5 simulations):


array([[10000.        , 10000.        , 10000.        , 10000.        ,
        10000.        ],
       [ 9985.23401362,  9967.10482175, 10064.6197046 , 10089.4998474 ,
        10061.93507151],
       [ 9958.72045047, 10121.8200447 , 10038.11339086, 10100.60080774,
        10056.13100986],
       [ 9957.85110915, 10082.38441895,  9859.78148407, 10038.99565093,
         9999.31469668],
       [ 9867.98624436,  9944.1848148 ,  9876.64182038, 10154.03348817,
        10041.43088466],
       [ 9922.60557827,  9957.77213379,  9842.47807504, 10114.86354143,
        10046.76721554]])


Monte Carlo Simulation Results shape: (1261, 1000)


## Integrate simulations with goals

### Subtask:
Add logic to compare the outcomes of the Monte Carlo simulations against the user's long-term investment goals to show the probability of achieving those goals.


**Reasoning**:
Implement the function to assess the probability of achieving the investment goal based on Monte Carlo simulation results and update the Streamlit UI to include input for the goal and display the assessment results.



In [26]:
import streamlit as st
import numpy as np
import pandas as pd # Ensure pandas is imported for DataFrame/Series if needed later

# Assume TERMS, call_gemini_api, display_strategy_builder, ASSET_CLASSES, historical_prices_df are defined in previous cells

# --- Goal Assessment Function ---
def assess_goal_feasibility(simulation_results: np.ndarray, investment_goal: float, num_trading_days: int):
    """
    Assesses the probability of achieving an investment goal based on Monte Carlo simulations.

    Args:
        simulation_results: A NumPy array from run_monte_carlo_simulation
                            with shape (num_trading_days + 1, num_simulations).
        investment_goal: The target investment amount.
        num_trading_days: The number of trading days simulated.

    Returns:
        A dictionary containing the probability of success, median final value,
        and 10th/90th percentile of final values.
    """
    if simulation_results.shape[0] != num_trading_days + 1:
         print(f"Warning: simulation_results shape mismatch. Expected {num_trading_days + 1}, got {simulation_results.shape[0]}.")
         # Handle potential mismatch, maybe return empty or error indication
         return {"probability_of_success": 0.0, "median_final_value": 0.0, "percentile_10": 0.0, "percentile_90": 0.0}


    # Extract the final simulated values (last row of the simulation results)
    final_values = simulation_results[-1, :]

    # Count how many simulations met or exceeded the goal
    successful_simulations_count = np.sum(final_values >= investment_goal)

    # Calculate the probability of achieving the goal
    total_simulations = simulation_results.shape[1]
    probability_of_success = (successful_simulations_count / total_simulations) * 100 if total_simulations > 0 else 0.0

    # Calculate relevant statistics for the final values
    median_final_value = np.median(final_values)
    percentile_10 = np.percentile(final_values, 10)
    percentile_90 = np.percentile(final_values, 90)

    return {
        "probability_of_success": probability_of_success,
        "median_final_value": median_final_value,
        "percentile_10": percentile_10,
        "percentile_90": percentile_90
    }

# --- UI Rendering Functions (Modified) ---

def display_portfolio_builder(historical_prices_df):
    """Renders the Portfolio Builder section."""
    st.header("📈 Portfolio Builder", divider='blue')
    st.info("""
    Plan your investment strategy by defining your initial investment,
    asset allocation, and long-term goals. Run simulations to assess feasibility.
    """, icon="📊")

    st.subheader("Investment Details")
    initial_investment = st.number_input(
        "Initial Investment Amount ($)",
        min_value=100.0,
        value=10000.0,
        step=100.0,
        format="%.2f",
        key='initial_investment_input'
    )

    st.subheader("Asset Allocation (%)")

    col1, col2, col3 = st.columns(3)

    with col1:
        stocks_allocation = st.slider("Stocks", 0, 100, 60, key='stocks_slider')
    with col2:
        bonds_allocation = st.slider("Bonds", 0, 100, 30, key='bonds_slider')
    with col3:
        other_allocation = st.slider("Other", 0, 100, 10, key='other_slider')

    asset_allocation = {
        'Stocks': stocks_allocation / 100.0,
        'Bonds': bonds_allocation / 100.0,
        'Other': other_allocation / 100.0
    }

    total_allocation = stocks_allocation + bonds_allocation + other_allocation

    if total_allocation > 100:
        st.warning(f"Total allocation exceeds 100% ({total_allocation}%). Please adjust.")
        allocation_ok = False
    elif total_allocation < 100:
         st.warning(f"Total allocation is less than 100% ({total_allocation}%). Please adjust.")
         allocation_ok = False
    else:
        st.success("Total allocation is 100%.")
        allocation_ok = True

    st.subheader("Goal Assessment")
    investment_goal = st.number_input(
        "Target Investment Goal Amount ($)",
        min_value=initial_investment,
        value=initial_investment * 2, # Default to doubling the investment
        step=100.0,
        format="%.2f",
        key='investment_goal_input'
    )

    # Assuming a fixed timeframe for the goal for simplicity based on simulation length
    # You could add an input for this if needed, but ensure it aligns with num_trading_days
    goal_timeframe_years = 5 # Matches the example simulation length
    num_trading_days = 252 * goal_timeframe_years # Approx trading days in the timeframe

    if st.button("Assess Goal Feasibility", type="primary", use_container_width=True):
        if not allocation_ok:
            st.error("Please fix the asset allocation before assessing the goal.")
        elif historical_prices_df.empty:
             st.error("Historical price data is not available. Cannot run simulations.")
        else:
            with st.spinner(f"Running {sample_num_simulations} simulations for {goal_timeframe_years} years..."):
                # Use sample parameters for simulation count for now
                monte_carlo_results = run_monte_carlo_simulation(
                    initial_investment,
                    asset_allocation,
                    historical_prices_df,
                    sample_num_simulations,
                    num_trading_days
                )

            if monte_carlo_results is not None and monte_carlo_results.shape[1] > 0:
                assessment_results = assess_goal_feasibility(monte_carlo_results, investment_goal, num_trading_days)

                st.subheader("Goal Feasibility Results")
                st.metric(label=f"Probability of reaching ${investment_goal:,.2f} in {goal_timeframe_years} years",
                          value=f"{assessment_results['probability_of_success']:.2f}%")

                st.write(f"Based on {sample_num_simulations} simulations over {goal_timeframe_years} years:")
                st.write(f"- Median estimated portfolio value: **${assessment_results['median_final_value']:,.2f}**")
                st.write(f"- 10th percentile estimated value: **${assessment_results['percentile_10']:,.2f}**")
                st.write(f"- 90th percentile estimated value: **${assessment_results['percentile_90']:,.2f}**")

                st.caption("Note: These projections are based on historical market data and are not guarantees of future performance.")
            else:
                st.error("Failed to run Monte Carlo simulations.")

    # Placeholder for displaying projected performance plot later
    # st.subheader("Projected Portfolio Performance")
    # Add plot here later


def main():
    """Main function to run the Streamlit app."""

    # --- App Header ---
    st.title("Financial Glossary & Portfolio Planner")
    st.markdown("Your interactive guide to Wealth Management & Private Banking terms and investment planning.")

    # --- AI Strategy Builder ---
    display_strategy_builder()

    # --- Portfolio Builder ---
    # Pass historical_prices_df to the portfolio builder function
    global historical_prices_df # Ensure historical_prices_df is accessible
    display_portfolio_builder(historical_prices_df)

    st.header("Glossary Terms", divider='gray')

    # --- Search Bar ---
    search_query = st.text_input("Search for a term...", "", placeholder="Type here to filter terms")

    # --- Filtered Glossary List ---
    filtered_terms = [
        term for term in TERMS if search_query.lower() in term['title'].lower()
    ]

    # The rest of the glossary display logic will go here later

# Ensure historical_prices_df is loaded before calling main
# Assuming the historical data loading script has been run and historical_prices_df exists globally
if 'historical_prices_df' in globals() and not historical_prices_df.empty:
    main()
else:
    st.error("Historical price data not loaded. Please run the data acquisition steps.")


2025-06-24 05:08:30.029 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:08:30.083 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-24 05:08:30.084 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:08:30.086 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:08:30.087 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:08:30.089 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:08:30.090 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:08:30.091 Thread 'MainThread': mi

## Display results

### Subtask:
Present the projected performance and the results of the Monte Carlo simulations in the Streamlit app, perhaps with charts and summary statistics.


**Reasoning**:
Modify the `display_portfolio_builder` function to display the projected historical performance chart and the Monte Carlo simulation results with a distribution plot.



In [27]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assume TERMS, call_gemini_api, display_strategy_builder, ASSET_CLASSES, historical_prices_df,
# project_historical_performance, run_monte_carlo_simulation, assess_goal_feasibility
# are defined in previous cells

# --- UI Rendering Functions (Modified) ---

def display_portfolio_builder(historical_prices_df):
    """Renders the Portfolio Builder section."""
    st.header("📈 Portfolio Builder", divider='blue')
    st.info("""
    Plan your investment strategy by defining your initial investment,
    asset allocation, and long-term goals. Run simulations to assess feasibility.
    """, icon="📊")

    st.subheader("Investment Details")
    initial_investment = st.number_input(
        "Initial Investment Amount ($)",
        min_value=100.0,
        value=10000.0,
        step=100.0,
        format="%.2f",
        key='initial_investment_input'
    )

    st.subheader("Asset Allocation (%)")

    col1, col2, col3 = st.columns(3)

    with col1:
        stocks_allocation = st.slider("Stocks", 0, 100, 60, key='stocks_slider')
    with col2:
        bonds_allocation = st.slider("Bonds", 0, 100, 30, key='bonds_slider')
    with col3:
        other_allocation = st.slider("Other", 0, 100, 10, key='other_slider')

    asset_allocation = {
        'Stocks': stocks_allocation / 100.0,
        'Bonds': bonds_allocation / 100.0,
        'Other': other_allocation / 100.0
    }

    total_allocation = stocks_allocation + bonds_allocation + other_allocation

    if total_allocation > 100:
        st.warning(f"Total allocation exceeds 100% ({total_allocation}%). Please adjust.")
        allocation_ok = False
    elif total_allocation < 100:
         st.warning(f"Total allocation is less than 100% ({total_allocation}%). Please adjust.")
         allocation_ok = False
    else:
        st.success("Total allocation is 100%.")
        allocation_ok = True

    st.subheader("Goal Assessment")
    investment_goal = st.number_input(
        "Target Investment Goal Amount ($)",
        min_value=initial_investment,
        value=initial_investment * 2, # Default to doubling the investment
        step=100.0,
        format="%.2f",
        key='investment_goal_input'
    )

    goal_timeframe_years = st.slider("Goal Timeframe (Years)", 1, 30, 5, key='goal_timeframe_slider')
    num_trading_days = 252 * goal_timeframe_years # Approx trading days per year

    # Use sample parameters for simulation count for now
    sample_num_simulations = 1000 # Keep a consistent number of simulations

    if st.button("Assess Goal Feasibility and Project Performance", type="primary", use_container_width=True):
        if not allocation_ok:
            st.error("Please fix the asset allocation before assessing the goal.")
        elif historical_prices_df.empty:
             st.error("Historical price data is not available. Cannot run simulations.")
        else:
            # --- Project Historical Performance ---
            st.subheader("Projected Portfolio Performance (Based on Historical Data)")
            with st.spinner("Projecting historical performance..."):
                 projected_values = project_historical_performance(
                     initial_investment,
                     asset_allocation,
                     historical_prices_df.tail(num_trading_days + 1) # Use data for the timeframe
                 )

            if not projected_values.empty:
                st.line_chart(projected_values)
            else:
                st.warning("Could not project historical performance with the current data and timeframe.")


            # --- Run Monte Carlo Simulation ---
            st.subheader(f"Monte Carlo Simulation Results ({sample_num_simulations} runs over {goal_timeframe_years} years)")
            with st.spinner(f"Running {sample_num_simulations} simulations..."):
                monte_carlo_results = run_monte_carlo_simulation(
                    initial_investment,
                    asset_allocation,
                    historical_prices_df, # Use full historical data for simulation parameters
                    sample_num_simulations,
                    num_trading_days
                )

            if monte_carlo_results is not None and monte_carlo_results.shape[1] > 0:
                assessment_results = assess_goal_feasibility(monte_carlo_results, investment_goal, num_trading_days)

                # --- Display Goal Feasibility Results ---
                st.metric(label=f"Probability of reaching ${investment_goal:,.2f} in {goal_timeframe_years} years",
                          value=f"{assessment_results['probability_of_success']:.2f}%")

                st.write(f"Based on {sample_num_simulations} simulations over {goal_timeframe_years} years:")
                st.write(f"- Median estimated portfolio value: **${assessment_results['median_final_value']:,.2f}**")
                st.write(f"- 10th percentile estimated value: **${assessment_results['percentile_10']:,.2f}**")
                st.write(f"- 90th percentile estimated value: **${assessment_results['percentile_90']:,.2f}**")

                # --- Visualize Simulation Results Distribution ---
                st.subheader("Distribution of Final Simulated Portfolio Values")
                final_values = monte_carlo_results[-1, :]

                fig, ax = plt.subplots()
                sns.histplot(final_values, kde=True, ax=ax)
                ax.set_title("Distribution of Final Portfolio Values")
                ax.set_xlabel("Final Portfolio Value ($)")
                ax.set_ylabel("Frequency")
                # Add vertical line for the goal
                ax.axvline(investment_goal, color='red', linestyle='dashed', linewidth=2, label=f'Goal: ${investment_goal:,.0f}')
                ax.legend()
                st.pyplot(fig)
                plt.close(fig) # Close the figure to prevent it from displaying automatically outside st.pyplot

                st.caption("Note: These projections are based on historical market data and are not guarantees of future performance.")
            else:
                st.error("Failed to run Monte Carlo simulations.")


def main():
    """Main function to run the Streamlit app."""

    # --- App Header ---
    st.title("Financial Glossary & Portfolio Planner")
    st.markdown("Your interactive guide to Wealth Management & Private Banking terms and investment planning.")

    # --- AI Strategy Builder ---
    display_strategy_builder()

    # --- Portfolio Builder ---
    # Pass historical_prices_df to the portfolio builder function
    global historical_prices_df # Ensure historical_prices_df is accessible
    if 'historical_prices_df' in globals() and not historical_prices_df.empty:
        display_portfolio_builder(historical_prices_df)
    else:
        st.error("Historical price data not loaded. Please run the data acquisition steps.")


    st.header("Glossary Terms", divider='gray')

    # --- Search Bar ---
    search_query = st.text_input("Search for a term...", "", placeholder="Type here to filter terms")

    # --- Filtered Glossary List ---
    filtered_terms = [
        term for term in TERMS if search_query.lower() in term['title'].lower()
    ]

    # The rest of the glossary display logic will go here later

# Ensure historical_prices_df is loaded before calling main
# Assuming the historical data loading script has been run and historical_prices_df exists globally
# main() # main will be called by the notebook runner

## Refactor and integrate

### Subtask:
Ensure the new code integrates smoothly with the existing glossary functionality and refactor the code for clarity and maintainability.


**Reasoning**:
Review the entire script for consistency, organize code logically, add comments and docstrings, verify existing functionality, remove redundant code, and prepare for testing. This involves modifying the existing code cell containing the main Streamlit application logic.



In [28]:
import streamlit as st
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Page Configuration ---
# This should be the first Streamlit command in your script.
st.set_page_config(
    page_title="Financial Glossary & Portfolio Planner",
    page_icon="✨",
    layout="centered",
    initial_sidebar_state="auto",
)

# --- Data: The definitions ---
# Storing the financial terms in a list of dictionaries.
TERMS = [
    {
        "title": "Convertibles (Convertible Bonds/Securities)",
        "definition": "These are bonds or preferred stocks that can be exchanged for a set number of common shares of the issuing company. They offer the potential for stock price appreciation along with the safety net of bond payments."
    },
    {
        "title": "Hedge",
        "definition": "In finance, a hedge is an investment made to lessen the potential losses (or gains) of another investment. It's like an insurance policy for your portfolio, designed to reduce overall risk."
    },
    {
        "title": "P/E Ratio (Price-to-Earnings Ratio)",
        "definition": "A common valuation metric, the P/E ratio compares a company's current stock price to its earnings per share. A higher P/E often signals that investors expect strong future growth or see the company as very stable.",
        "formula": r"$$ \text{P/E Ratio} = \frac{\text{Market Price per Share}}{\text{Earnings per Share (EPS)}} $$"
    },
    {
        "title": "Vintage Year (Private Equity)",
        "definition": "This refers to the year a private equity fund first started investing its committed capital. It's crucial for comparing fund performance, as market conditions vary significantly from year to year."
    },
    {
        "title": "Tax-Aware Borrowing",
        "definition": "This is a borrowing strategy where you consider the tax implications of the borrowed money and how it's used. The goal is often to maximize tax deductions on interest or to fund investments that offer tax advantages."
    },
    {
        "title": "Draw-down (Private Equity)",
        "definition": 'Also known as a "capital call," this is when a private equity fund requests a portion of the capital that investors have pledged. The fund then uses this money to make new investments in companies.'
    },
    {
        "title": "Dodd-Frank",
        "definition": 'Enacted in 2010 after the 2008 financial crisis, this comprehensive law (Dodd-Frank Wall Street Reform and Consumer Protection Act) aims to stabilize the financial system by increasing accountability and transparency, preventing "too big to fail" scenarios, protecting taxpayers from bailouts, and safeguarding consumers.'
    },
    {
        "title": "Research International Markets",
        "definition": "This involves gathering and analyzing information about financial markets outside an investor's home country. It includes studying economic conditions, political stability, regulations, and specific companies or industries to find global investment opportunities and assess risks."
    },
    {
        "title": "RoE (Return on Equity)",
        "definition": "A profitability measure that shows how much profit a company generates for each dollar of shareholder investment. A higher RoE generally indicates efficient use of shareholder funds to create profits.",
        "formula": r"$$ \text{RoE} = \frac{\text{Net Income}}{\text{Shareholders' Equity}} $$"
    },
    {
        "title": "Loan-to-Value (LTV)",
        "definition": "This ratio compares the size of a loan to the appraised value of the asset being financed. Lenders use LTV to assess risk; a higher LTV means higher risk.",
        "formula": r"$$ \text{LTV} = \frac{\text{Loan Amount}}{\text{Appraised Value of Asset}} \times 100\% $$"
    },
    {
        "title": "Subscription Lines",
        "definition": "These are short-term credit lines (also known as Subscription Facilities or Subscription Credit Facilities) banks provide to private equity funds. They allow fund managers to access capital quickly for investments while waiting for committed funds from investors to arrive, offering flexibility and speed."
    },
    {
        "title": "Barbell-Structure (Barbell Strategy)",
        "definition": "An investment approach where a portfolio is heavily weighted at two extremes, with minimal or no allocation in the middle. For example, in bonds, it might mean investing in both very short-term and very long-term bonds while avoiding medium-term ones. This strategy often balances liquidity and high returns or manages specific risks."
    },
    {
        "title": "Parametric Tax-Loss Harvesting Strategy",
        "definition": "This is an automated or systematic method for tax-loss harvesting. It uses predefined rules to identify and sell investments at a loss, offsetting capital gains and potentially lowering taxable income. It aims for continuous tax efficiency."
    },
    {
        "title": "Derivatives and Structured Notes",
        "definition": "**Derivatives:** Financial contracts whose value comes from an underlying asset, like a stock, index, or commodity. Examples include options, futures, and swaps, used for hedging, speculation, or arbitrage.\n\n**Structured Notes:** Debt securities, typically issued by banks, whose returns are linked to the performance of an underlying asset or index. They are customized to meet specific risk/return goals but can be complex and carry unique risks."
    },
    {
        "title": "Private Credit",
        "definition": "This refers to debt financing provided directly by non-bank lenders (such as private credit funds) to companies, offering an alternative to traditional bank loans or public bonds. It includes various strategies like direct lending and mezzanine financing."
    },
    {
        "title": "Recovery Rates",
        "definition": "In the context of defaulted debt, the recovery rate is the percentage of the original principal and interest that a lender or bondholder manages to recover after a default, through actions like bankruptcy proceedings or asset sales. It's the inverse of the loss given default."
    },
    {
        "title": "Delaware Trust",
        "definition": "A trust established under Delaware law, known for its favorable trust statutes. Delaware trusts offer advantages like perpetual duration, strong asset protection, privacy, and flexibility in their structure, making them popular for sophisticated estate planning."
    },
    {
        "title": "GRAT (Grantor Retained Annuity Trust)",
        "definition": "An estate planning tool used to transfer assets to beneficiaries with reduced gift and estate taxes. The person who creates the GRAT (the grantor) receives annuity payments for a set period. If the trust assets grow faster than a specific IRS-assumed rate, the excess appreciation passes to the beneficiaries tax-free."
    },
    {
        "title": "Stock Options",
        "definition": 'A contract that gives the holder the right, but not the obligation, to buy or sell a specific number of company shares at a set price (the "strike price") within a certain timeframe. They are often used as employee compensation or traded in markets.'
    },
    {
        "title": "Wirehouse",
        "definition": "Historically, this term referred to large, full-service brokerage firms with extensive branch networks and many financial advisors. They offer a broad range of services, including brokerage, investment banking, and wealth management. Prominent examples include Merrill Lynch and Morgan Stanley."
    },
    {
        "title": "Wrap Accounts",
        "definition": 'An investment advisory account where clients pay a single, all-inclusive fee (usually a percentage of aum) that "wraps" together advisory fees, brokerage commissions, transaction costs, and sometimes custodial fees, simplifying pricing for clients.'
    },
    {
        "title": "CET1 Ratio (Common Equity Tier 1 Ratio)",
        "definition": "A critical measure of a bank's financial health, it compares a bank's highest quality capital (common equity tier 1) to its total risk-weighted assets. A higher CET1 ratio indicates stronger capital and greater resilience to financial shocks, and it's closely monitored by regulators."
    }
]

# --- Gemini API Call Function ---
@st.cache_data(show_spinner=False) # Cache the response to avoid re-calling the API
def call_gemini_api(prompt, response_type='text'):
    """
    Calls the Gemini API to generate text based on a prompt.

    Args:
        prompt: The text prompt to send to the API.
        response_type: 'text' or 'json'. Specifies the desired output format.

    Returns:
        The API response as a string or parsed JSON, or an error message.
    """
    # API key handling should ideally be done more securely in a production app
    # For this example, we assume it's handled by the environment or Streamlit secrets
    api_key = ""  # Placeholder, replace with actual key retrieval if not using secrets/env
    # In a real app, use st.secrets["GEMINI_API_KEY"] or environment variables

    # Check if API key is available (basic check)
    if not api_key:
         # If running in a notebook without secrets/env, you might need to set it here temporarily for testing
         # print("Warning: GEMINI_API_KEY not found. AI Strategy Builder will not work.")
         return "Error: API key for AI Strategy Builder is not configured."


    api_url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent?key={api_key}"

    payload = {
        "contents": [{"role": "user", "parts": [{"text": prompt}]}],
        "generationConfig": {"temperature": 0.5, "maxOutputTokens": 1024}
    }

    if response_type == 'json':
        payload["generationConfig"]["responseMimeType"] = "application/json"
        # Define the expected JSON schema for the response
        payload["generationConfig"]["responseSchema"] = {
            "type": "OBJECT",
            "properties": {
                "question": {"type": "STRING"},
                "options": {"type": "ARRAY", "items": {"type": "STRING"}},
                "correctAnswerIndex": {"type": "INTEGER"},
                "explanation": {"type": "STRING"}
            },
            "required": ["question", "options", "correctAnswerIndex", "explanation"]
        }

    try:
        response = requests.post(api_url, headers={'Content-Type': 'application/json'}, json=payload)
        response.raise_for_status()  # Will raise an exception for 4xx/5xx errors

        if response_type == 'json':
            return response.json()
        else:
             result = response.json()
             content = result.get('candidates', [{}])[0].get('content', {}).get('parts', [{}])[0].get('text')
             if not content:
                 return "Error: Received an empty text response from the API."
             return content

    except requests.exceptions.RequestException as e:
        return f"Error calling API: {e}"
    except (KeyError, IndexError, json.JSONDecodeError) as e:
        return f"Error parsing API response: {e}"


# --- Data Acquisition (Assuming this is run before main) ---
# Define asset classes and their representative tickers
ASSET_CLASSES = {
    "Stocks": "^GSPC",  # S&P 500
    "Bonds": "AGG",     # iShares Core U.S. Aggregate Bond ETF
    "Other": "GLD"      # Gold
}

# Define the date range for historical data
START_DATE = "2005-01-01"
END_DATE = pd.to_datetime("today").strftime("%Y-%m-%d")

# Download historical data - This part should ideally be in a separate setup cell
# For integration, we'll assume historical_prices_df is already loaded globally
# import yfinance as yf # Import yfinance if not already imported

# historical_data = {}
# for asset_name, ticker in ASSET_CLASSES.items():
#     try:
#         data = yf.download(ticker, start=START_DATE, end=END_DATE, auto_adjust=True, progress=False)
#         if not data.empty and 'Close' in data.columns:
#             historical_data[asset_name] = data['Close']
#         else:
#             print(f"Warning: No data downloaded or 'Close' column missing for {asset_name} ({ticker})")
#     except Exception as e:
#         print(f"Error downloading data for {asset_name} ({ticker}): {e}")

# if historical_data:
#     historical_prices_df = pd.concat(historical_data.values(), axis=1, keys=historical_data.keys())
#     historical_prices_df.fillna(method='ffill', inplace=True)
#     historical_prices_df.dropna(inplace=True)
# else:
#     historical_prices_df = pd.DataFrame()
#     print("No historical data could be downloaded successfully for any asset.")


# --- Portfolio Calculation Functions ---

def project_historical_performance(initial_investment: float, asset_allocation: dict, historical_prices_df: pd.DataFrame) -> pd.Series:
    """
    Projects potential portfolio performance over time based on historical data.

    Args:
        initial_investment: The initial amount invested.
        asset_allocation: A dictionary where keys are asset class names
                          and values are their percentage allocations (e.g., {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}).
        historical_prices_df: DataFrame with historical daily closing prices
                              for each asset class, indexed by date.

    Returns:
        A pandas Series containing the projected portfolio value over time, indexed by date.
        Returns an empty Series if calculation is not possible.
    """
    if historical_prices_df.empty:
        return pd.Series()

    # Ensure asset allocation sums to 1 (or close to it due to floating point)
    total_allocation = sum(asset_allocation.values())
    if not (0.99 <= total_allocation <= 1.01):
        if total_allocation != 0:
             asset_allocation = {asset: allocation / total_allocation for asset, allocation in asset_allocation.items()}
        else:
            print("Warning: Total asset allocation is 0%. Cannot project performance.")
            return pd.Series(index=historical_prices_df.index, data=initial_investment)


    # Calculate daily returns for each asset class
    # Use .copy() to avoid SettingWithCopyWarning
    daily_returns = historical_prices_df.pct_change().dropna().copy()


    # Calculate weighted daily return for the portfolio
    weighted_returns_df = pd.DataFrame(0.0, index=daily_returns.index, columns=daily_returns.columns)

    for asset, allocation in asset_allocation.items():
        if asset in daily_returns.columns:
            weighted_returns_df[asset] = daily_returns[asset] * allocation
        else:
            print(f"Warning: Asset '{asset}' not found in historical data for performance projection.")

    # Sum the weighted returns across assets for each day to get the portfolio's daily return
    weighted_daily_returns = weighted_returns_df.sum(axis=1)

    # Calculate cumulative return of the portfolio (starting from 1 for initial investment)
    # Add a 1 at the beginning of the series to represent the initial state (1 + 0% return)
    cumulative_returns = (1 + weighted_daily_returns).cumprod()

    # Project the portfolio value over time
    projected_portfolio_value = initial_investment * cumulative_returns

    # The cumulative product starts from the second day. Add the initial investment for the first day.
    # Find the first date in the original historical data index
    first_date = historical_prices_df.index[0]
    # Create a Series for the first day with the initial investment
    initial_value_series = pd.Series(initial_investment, index=[first_date])

    # Concatenate the initial value with the projected values
    # Use .reindex to ensure the initial date is included if it wasn't in cumulative_returns index
    full_projected_portfolio_value = pd.concat([initial_value_series, projected_portfolio_value]).sort_index()


    return full_projected_portfolio_value


def run_monte_carlo_simulation(initial_investment: float, asset_allocation: dict, historical_prices_df: pd.DataFrame, num_simulations: int, num_trading_days: int) -> np.ndarray:
    """
    Runs Monte Carlo simulations to project potential portfolio performance.

    Args:
        initial_investment: The initial amount invested.
        asset_allocation: A dictionary where keys are asset class names
                          and values are their percentage allocations (e.g., {'Stocks': 0.6, 'Bonds': 0.3, 'Other': 0.1}).
        historical_prices_df: DataFrame with historical daily closing prices
                              for each asset class, indexed by date. Used to calculate
                              mean and std dev of historical returns.
        num_simulations: The number of Monte Carlo simulations to run.
        num_trading_days: The number of trading days to simulate into the future.

    Returns:
        A NumPy array where each column represents a simulated portfolio value
        over the specified number of trading days, and each row represents
        a specific time step (day). The shape will be (num_trading_days + 1, num_simulations).
        Returns an empty NumPy array if simulation is not possible.
    """
    if historical_prices_df.empty:
        return np.array([])

    # Ensure asset allocation sums to 1 (or close to it due to floating point)
    total_allocation = sum(asset_allocation.values())
    if not (0.99 <= total_allocation <= 1.01):
        if total_allocation != 0:
             asset_allocation = {asset: allocation / total_allocation for asset, allocation in asset_allocation.items()}
        else:
            print("Warning: Total asset allocation is 0%. Cannot run simulation.")
            return np.full((num_trading_days + 1, num_simulations), initial_investment)


    # Calculate daily returns for each asset class
    daily_returns = historical_prices_df.pct_change().dropna()


    # Calculate weighted daily return for the portfolio based on historical data
    weighted_returns_df = pd.DataFrame(0.0, index=daily_returns.index, columns=daily_returns.columns)

    for asset, allocation in asset_allocation.items():
        if asset in daily_returns.columns:
            weighted_returns_df[asset] = daily_returns[asset] * allocation
        else:
             print(f"Warning: Asset '{asset}' not found in historical data for simulation parameters.")


    # Get the historical daily returns for the portfolio
    historical_portfolio_daily_returns = weighted_returns_df.sum(axis=1)

    # Calculate the mean and standard deviation of historical daily returns
    if historical_portfolio_daily_returns.empty:
         print("Error: Historical portfolio daily returns are empty. Cannot run simulation.")
         return np.array([])

    mean_daily_return = historical_portfolio_daily_returns.mean()
    std_daily_return = historical_portfolio_daily_returns.std()

    # Array to store simulation results
    simulation_results = np.zeros((num_trading_days + 1, num_simulations))
    simulation_results[0, :] = initial_investment # Set the initial investment for all simulations

    # Run Monte Carlo simulations
    for day in range(1, num_trading_days + 1):
        # Generate random daily returns for all simulations for the current day
        # Draw from a normal distribution based on historical mean and std dev
        random_daily_returns = np.random.normal(mean_daily_return, std_daily_return, num_simulations)

        # Calculate portfolio value for the current day for all simulations
        simulation_results[day, :] = simulation_results[day - 1, :] * (1 + random_daily_returns)

    return simulation_results


def assess_goal_feasibility(simulation_results: np.ndarray, investment_goal: float) -> dict:
    """
    Assesses the probability of achieving an investment goal based on Monte Carlo simulations.

    Args:
        simulation_results: A NumPy array from run_monte_carlo_simulation
                            with shape (num_trading_days + 1, num_simulations).
        investment_goal: The target investment amount.

    Returns:
        A dictionary containing the probability of success, median final value,
        and 10th/90th percentile of final values. Returns an empty dictionary if simulation_results is empty.
    """
    if simulation_results.size == 0:
        return {"probability_of_success": 0.0, "median_final_value": 0.0, "percentile_10": 0.0, "percentile_90": 0.0}

    # Extract the final simulated values (last row of the simulation results)
    final_values = simulation_results[-1, :]

    # Count how many simulations met or exceeded the goal
    successful_simulations_count = np.sum(final_values >= investment_goal)

    # Calculate the probability of achieving the goal
    total_simulations = simulation_results.shape[1]
    probability_of_success = (successful_simulations_count / total_simulations) * 100 if total_simulations > 0 else 0.0

    # Calculate relevant statistics for the final values
    median_final_value = np.median(final_values)
    percentile_10 = np.percentile(final_values, 10)
    percentile_90 = np.percentile(final_values, 90)

    return {
        "probability_of_success": probability_of_success,
        "median_final_value": median_final_value,
        "percentile_10": percentile_10,
        "percentile_90": percentile_90
    }


# --- UI Rendering Functions ---

def display_strategy_builder():
    """Renders the AI Strategy Builder section."""
    st.header("✨ AI Strategy Builder", divider='rainbow')
    st.info("""
    Describe an investment goal below. The AI will generate a conceptual strategy,
    incorporating relevant terms from the glossary.
    """, icon="💡")

    goal_input = st.text_area(
        "Describe your investment goal:",
        placeholder="e.g., I want high growth over 10 years and I'm okay with some risk.",
        height=100,
        key='strategy_goal_input'
    )

    if st.button("Generate Strategy Concept", type="primary", use_container_width=True, key='generate_strategy_button'):
        if not goal_input.strip():
            st.warning("Please describe your investment goal.")
        else:
            term_titles = ", ".join([term['title'] for term in TERMS])
            prompt = (f"An investor has the following goal: \"{goal_input}\". Generate a high-level, conceptual "
                      f"investment strategy for them. This is for educational purposes only and is not financial advice. "
                      f"In your response, please use relevant terms from the following list and wrap them in double "
                      f"asterisks (e.g., **Term Title**). List of available terms: {term_titles}.")

            with st.spinner("Generating strategy..."):
                strategy = call_gemini_api(prompt)

            st.subheader("Generated Strategy Concept")
            st.markdown(strategy)
            st.caption("Disclaimer: This is an AI-generated educational concept and not financial advice. "
                       "Consult a qualified professional before making any investment decisions.")


def display_portfolio_builder(historical_prices_df):
    """
    Renders the Portfolio Builder section, allowing users to input investment details,
    define asset allocation, set goals, and run simulations.

    Args:
        historical_prices_df: DataFrame with historical daily closing prices
                              for each asset class, indexed by date.
    """
    st.header("📈 Portfolio Builder", divider='blue')
    st.info("""
    Plan your investment strategy by defining your initial investment,
    asset allocation, and long-term goals. Run simulations to assess feasibility.
    """, icon="📊")

    st.subheader("Investment Details")
    initial_investment = st.number_input(
        "Initial Investment Amount ($)",
        min_value=100.0,
        value=10000.0,
        step=100.0,
        format="%.2f",
        key='pb_initial_investment_input' # Use unique key
    )

    st.subheader("Asset Allocation (%)")

    col1, col2, col3 = st.columns(3)

    with col1:
        stocks_allocation = st.slider("Stocks", 0, 100, 60, key='pb_stocks_slider') # Use unique key
    with col2:
        bonds_allocation = st.slider("Bonds", 0, 100, 30, key='pb_bonds_slider')   # Use unique key
    with col3:
        other_allocation = st.slider("Other", 0, 100, 10, key='pb_other_slider')   # Use unique key

    asset_allocation = {
        'Stocks': stocks_allocation / 100.0,
        'Bonds': bonds_allocation / 100.0,
        'Other': other_allocation / 100.0
    }

    total_allocation = stocks_allocation + bonds_allocation + other_allocation

    if total_allocation > 100:
        st.warning(f"Total allocation exceeds 100% ({total_allocation}%). Please adjust.")
        allocation_ok = False
    elif total_allocation < 100:
         st.warning(f"Total allocation is less than 100% ({total_allocation}%). Please adjust.")
         allocation_ok = False
    else:
        st.success("Total allocation is 100%.")
        allocation_ok = True

    st.subheader("Goal Assessment")
    investment_goal = st.number_input(
        "Target Investment Goal Amount ($)",
        min_value=initial_investment,
        value=initial_investment * 2, # Default to doubling the investment
        step=100.0,
        format="%.2f",
        key='pb_investment_goal_input' # Use unique key
    )

    goal_timeframe_years = st.slider("Goal Timeframe (Years)", 1, 30, 5, key='pb_goal_timeframe_slider') # Use unique key
    num_trading_days = 252 * goal_timeframe_years # Approx trading days per year

    # Use a fixed number of simulations for consistency
    num_simulations = 1000

    if st.button("Assess Goal Feasibility and Project Performance", type="primary", use_container_width=True, key='pb_run_simulation_button'): # Use unique key
        if not allocation_ok:
            st.error("Please fix the asset allocation before assessing the goal.")
        elif historical_prices_df.empty:
             st.error("Historical price data is not available. Cannot run simulations.")
        elif num_trading_days == 0:
             st.error("Goal timeframe must be at least 1 year.")
        else:
            # --- Project Historical Performance ---
            st.subheader("Projected Portfolio Performance (Based on Historical Data)")
            # Use historical data slice corresponding to the goal timeframe length
            historical_slice = historical_prices_df.tail(num_trading_days + 1)

            if not historical_slice.empty and len(historical_slice) > 1: # Need at least 2 data points for pct_change
                 with st.spinner("Projecting historical performance..."):
                      projected_values = project_historical_performance(
                          initial_investment,
                          asset_allocation,
                          historical_slice
                      )

                 if not projected_values.empty:
                     # Reindex the projected values to have a continuous date range for the plot
                     # This prevents gaps if the historical slice had missing dates (unlikely after ffill/dropna but good practice)
                     # Create a date range covering the slice duration
                     date_range = pd.date_range(start=historical_slice.index[0], end=historical_slice.index[-1], freq='D')
                     projected_values_reindexed = projected_values.reindex(date_range).fillna(method='ffill')

                     st.line_chart(projected_values_reindexed)
                 else:
                     st.warning("Could not project historical performance with the current data and timeframe.")
            else:
                 st.warning("Not enough historical data available for the selected timeframe to project performance.")


            # --- Run Monte Carlo Simulation ---
            st.subheader(f"Monte Carlo Simulation Results ({num_simulations} runs over {goal_timeframe_years} years)")
            with st.spinner(f"Running {num_simulations} simulations..."):
                monte_carlo_results = run_monte_carlo_simulation(
                    initial_investment,
                    asset_allocation,
                    historical_prices_df, # Use full historical data for simulation parameters
                    num_simulations,
                    num_trading_days
                )

            if monte_carlo_results is not None and monte_carlo_results.size > 0:
                assessment_results = assess_goal_feasibility(monte_carlo_results, investment_goal)

                # --- Display Goal Feasibility Results ---
                st.metric(label=f"Probability of reaching ${investment_goal:,.2f} in {goal_timeframe_years} years",
                          value=f"{assessment_results['probability_of_success']:.2f}%")

                st.write(f"Based on {num_simulations} simulations over {goal_timeframe_years} years:")
                st.write(f"- Median estimated portfolio value: **${assessment_results['median_final_value']:,.2f}**")
                st.write(f"- 10th percentile estimated value: **${assessment_results['percentile_10']:,.2f}**")
                st.write(f"- 90th percentile estimated value: **${assessment_results['percentile_90']:,.2f}**")

                # --- Visualize Simulation Results Distribution ---
                st.subheader("Distribution of Final Simulated Portfolio Values")
                final_values = monte_carlo_results[-1, :]

                if final_values.size > 0:
                    fig, ax = plt.subplots()
                    sns.histplot(final_values, kde=True, ax=ax)
                    ax.set_title("Distribution of Final Portfolio Values")
                    ax.set_xlabel("Final Portfolio Value ($)")
                    ax.set_ylabel("Frequency")
                    # Add vertical line for the goal
                    ax.axvline(investment_goal, color='red', linestyle='dashed', linewidth=2, label=f'Goal: ${investment_goal:,.0f}')
                    ax.legend()
                    st.pyplot(fig)
                    plt.close(fig) # Close the figure to prevent it from displaying automatically outside st.pyplot
                else:
                    st.warning("No simulation results to display the distribution.")

                st.caption("Note: These projections are based on historical market data and are not guarantees of future performance.")
            else:
                st.error("Failed to run Monte Carlo simulations. Check historical data and parameters.")


def display_glossary(filtered_terms):
     """
     Renders the Glossary section, displaying filtered terms.

     Args:
         filtered_terms: A list of dictionaries representing the financial terms
                         to display, filtered by the search query.
     """
     st.header("Glossary Terms", divider='gray')

     # --- Search Bar ---
     search_query = st.text_input("Search for a term...", "", placeholder="Type here to filter terms", key='glossary_search_input') # Use unique key

     # --- Filtered Glossary List ---
     current_filtered_terms = [
         term for term in TERMS if search_query.lower() in term['title'].lower()
     ]

     if not current_filtered_terms:
         st.info("No terms match your search query.")
     else:
         for term in current_filtered_terms:
             with st.expander(f"**{term['title']}**"):
                 st.write(term["definition"])
                 if "formula" in term:
                     st.markdown(term["formula"])


def main():
    """Main function to run the Streamlit app."""

    # --- App Header ---
    st.title("Financial Glossary & Portfolio Planner")
    st.markdown("Your interactive guide to Wealth Management & Private Banking terms and investment planning.")

    # --- AI Strategy Builder ---
    display_strategy_builder()

    # --- Portfolio Builder ---
    # Ensure historical_prices_df is loaded before displaying the portfolio builder
    global historical_prices_df # Access the global variable
    if 'historical_prices_df' in globals() and not historical_prices_df.empty:
        display_portfolio_builder(historical_prices_df)
    else:
        st.error("Historical price data not loaded. Please run the data acquisition steps.")

    # --- Glossary ---
    # The glossary filtering happens inside the display_glossary function now
    display_glossary(TERMS) # Pass the full list, filtering is done internally


# Ensure historical_prices_df is loaded before calling main
# Assuming the historical data loading script has been run and historical_prices_df exists globally
# The notebook runner will execute cells sequentially, so the data loading cell should run before this one.
# We add a check in main and display_portfolio_builder just in case.
# main() # main will be called by the notebook runner environment

2025-06-24 05:09:51.033 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-24 05:09:51.035 No runtime found, using MemoryCacheStorageManager


## Summary:

### Data Analysis Key Findings

*   The Streamlit application was successfully enhanced with a "Portfolio Builder" section, including inputs for initial investment, asset allocation (Stocks, Bonds, Other), and investment goals.
*   Basic validation ensures that the asset allocation percentages sum to 100%.
*   Historical price data for representative tickers (^GSPC, AGG, GLD) was successfully acquired using `yfinance` and processed into a pandas DataFrame, handling missing values.
*   Functions were developed to project historical portfolio performance and run Monte Carlo simulations based on user inputs and historical data.
*   Logic was implemented to assess the feasibility of achieving a user-defined investment goal by analyzing the outcomes of the Monte Carlo simulations, calculating the probability of success and key statistics (median, 10th/90th percentiles).
*   The projected historical performance and Monte Carlo simulation results, including summary statistics and a distribution plot, are displayed in the Streamlit UI.
*   The new portfolio builder features were integrated into the existing Streamlit application alongside the glossary functionality, with code refactoring for improved clarity, organization, and maintainability.

### Insights or Next Steps

*   Enhance the Monte Carlo simulation by allowing users to select different historical periods or use alternative models for generating simulated returns (e.g., GARCH models) to explore a wider range of potential market behaviors.
*   Add features for portfolio rebalancing over time within the simulations or projections to provide a more realistic view of long-term performance under different strategies.


In [ ]:
!streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.193.6:8501

